# **JOWMDroid**

Método de detecção de aplicativos maliciosos Android com base na ponderação de recursos com otimização conjunta de mapeamento de peso e parâmetros de classificação.

## Importações

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression

# métricas
from sklearn.model_selection import cross_val_score

# modelos
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# tirando limitações de exibição de linhas e colunas do dataset
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Datasets

Drebin

In [3]:
drebin = pd.read_csv("Drebin215.csv", sep=";")

In [4]:
drebin['class'].replace('B', 0, inplace=True)
drebin['class'].replace('S', 1, inplace=True)

In [5]:
drebin.head()

,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,android.permission.SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,android.telephony.SmsManager,android.permission.READ_PHONE_STATE,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Ljava.lang.Class.getField,Landroid.content.Context.unregisterReceiver,android.permission.GET_ACCOUNTS,android.permission.RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,android.permission.READ_SMS,getCallingUid,Ljavax.crypto.spec.SecretKeySpec,android.intent.action.BOOT_COMPLETED,android.permission.USE_CREDENTIALS,android.permission.MANAGE_ACCOUNTS,android.content.pm.PackageInfo,KeySpec,TelephonyManager.getLine1Number,DexClassLoader,HttpGet.init,SecretKey,Ljava.lang.Class.getMethod,System.loadLibrary,android.intent.action.SEND,Ljavax.crypto.Cipher,android.permission.WRITE_SMS,android.permission.READ_SYNC_SETTINGS,android.permission.AUTHENTICATE_ACCOUNTS,android.telephony.gsm.SmsManager,com.android.browser.permission.WRITE_HISTORY_BOOKMARKS,TelephonyManager.getSubscriberId,mount,android.permission.INSTALL_PACKAGES,Runtime.getRuntime,android.permission.CAMERA,Ljava.lang.Object.getClass,android.permission.WRITE_SYNC_SETTINGS,com.android.browser.permission.READ_HISTORY_BOOKMARKS,Ljava.lang.Class.forName,android.permission.INTERNET,android.intent.action.PACKAGE_REPLACED,Binder,android.intent.action.SEND_MULTIPLE,android.permission.RECORD_AUDIO,IBinder,android.os.IBinder,createSubprocess,android.permission.NFC,android.permission.ACCESS_LOCATION_EXTRA_COMMANDS,URLClassLoader,android.permission.WRITE_APN_SETTINGS,abortBroadcast,android.permission.BIND_REMOTEVIEWS,android.intent.action.TIME_SET,android.permission.READ_PROFILE,TelephonyManager.getDeviceId,android.permission.MODIFY_AUDIO_SETTINGS,getCallingPid,android.permission.READ_SYNC_STATS,android.permission.BROADCAST_STICKY,android.intent.action.PACKAGE_REMOVED,android.intent.action.TIMEZONE_CHANGED,android.permission.WAKE_LOCK,android.permission.RECEIVE_BOOT_COMPLETED,android.permission.RESTART_PACKAGES,Ljava.lang.Class.getPackage,chmod,Ljava.lang.Class.getDeclaredClasses,android.intent.action.ACTION_POWER_DISCONNECTED,android.intent.action.PACKAGE_ADDED,PathClassLoader,TelephonyManager.getSimSerialNumber,Runtime.load,TelephonyManager.getCallState,android.permission.BLUETOOTH,android.permission.READ_CALENDAR,android.permission.READ_CALL_LOG,android.permission.SUBSCRIBED_FEEDS_WRITE,android.permission.READ_EXTERNAL_STORAGE,TelephonyManager.getSimCountryIso,sendMultipartTextMessage,PackageInstaller,android.permission.VIBRATE,remount,android.intent.action.ACTION_SHUTDOWN,sendDataMessage,android.permission.ACCESS_NETWORK_STATE,chown,HttpPost.init,Ljava.lang.Class.getClasses,android.permission.SUBSCRIBED_FEEDS_READ,TelephonyManager.isNetworkRoaming,android.permission.CHANGE_WIFI_MULTICAST_STATE,android.permission.WRITE_CALENDAR,android.intent.action.PACKAGE_DATA_CLEARED,android.permission.MASTER_CLEAR,HttpUriRequest,android.permission.UPDATE_DEVICE_STATS,android.permission.WRITE_CALL_LOG,android.permission.DELETE_PACKAGES,android.permission.GET_TASKS,android.permission.GLOBAL_SEARCH,android.permission.DELETE_CACHE_FILES,android.permission.WRITE_USER_DICTIONARY,android.intent.action.PACKAGE_CHANGED,android.intent.action.NEW_OUTGOING_CALL,android.permission.REORDER_TASKS,android.permission.WRITE_PROFILE,android.permission.SET_WALLPAPER,android.permission.BIND_INPUT_METHOD,divideMessage,android.permission.READ_SOCIAL_STREAM,android.permission.READ_USER_DICTIONARY,android.permission.PROCESS_OUTGOING_CALLS,android.permission.CALL_PRIVILEGED,Runtime.exec,android.permission.BIND_WALLPAPER,android.permission.RECEIVE_WAP_PUSH,android.permission.DUMP,android.permission.BATTERY_STATS,android.permission.ACCESS_COARSE_LOCATION,android.permission.SET_TIME,android.intent.action.SENDTO,android.permission.WRITE_SOCIAL_STREAM,an

## Mutual Information (IG)

Utilizamos o mutual_info_regression como método de Ganho de Informação (IG) para selecionar as características mais relevantes para o modelo. A nota de corte é de 0.05.

In [6]:
x_aux = drebin.drop(columns=["class"])
y_aux = drebin["class"]

In [7]:
def calculate_mi(x, y):
    columns = x.columns
    mi = mutual_info_regression(x, y)
    return pd.Series(mi, index=np.array(columns)).sort_values(ascending=False)

In [8]:
mi = calculate_mi(x_aux, y_aux)
mi

transact                                                     0.187831
bindService                                                  0.177366
attachInterface                                              0.175732
onServiceConnected                                           0.171959
android.os.Binder                                            0.171474
ServiceConnection                                            0.170820
android.permission.SEND_SMS                                  0.157275
Ljava.lang.Class.getMethods                                  0.126805
Ljava.lang.Class.getCanonicalName                            0.125938
Ljava.net.URLDecoder                                         0.110631
android.content.pm.Signature                                 0.107098
Ljava.lang.Class.cast                                        0.099083
android.permission.READ_PHONE_STATE                          0.098328
android.telephony.SmsManager                                 0.095751
getBinder           

In [9]:
drebin_mi = pd.DataFrame()
drebin_mi = drebin[["transact",
"attachInterface",
"onServiceConnected",
"bindService",
"android.os.Binder",
"ServiceConnection",
"android.permission.SEND_SMS",
"Ljava.lang.Class.getCanonicalName",
"Ljava.lang.Class.cast",
"Ljava.lang.Class.getMethods",
"Ljava.net.URLDecoder",
"android.content.pm.Signature",
"android.telephony.SmsManager",
"android.permission.READ_PHONE_STATE",
"ClassLoader",
"getBinder",
"Ljava.lang.Class.getField",
"android.permission.READ_SMS",
"android.permission.RECEIVE_SMS",
"Landroid.content.Context.unregisterReceiver",
"Landroid.content.Context.registerReceiver",
"android.permission.GET_ACCOUNTS",
"Ljava.lang.Class.getDeclaredField",
"getCallingUid",
"HttpGet.init",
"Ljava.lang.Class.getMethod",
"Ljavax.crypto.spec.SecretKeySpec",
"android.intent.action.BOOT_COMPLETED",
"class"]]

In [10]:
drebin_mi.shape

(15036, 29)

## Separando conjunto de dados

Variáveis alvo e preditoras

In [11]:
x = drebin_mi.drop(columns=["class"])
y = drebin_mi["class"]

Variáveis de treino e teste

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

## Modelos para mapear pesos

Utilizamos esses três modelos abaixo por causa de seus métodos de seleção de características, que através de seus cálculos, distribuem pesos para as características. Após cada modelo distribuir os pesos das características, normalizamos os pesos entre 0 e 1 para todos os modelos e em seguida tiramos a média.

In [13]:
# SVM
model_SVM = SVC(kernel='linear')

# Random Forest
model_RF = RandomForestClassifier()

# Logistic Regression
model_LR = LogisticRegression()

Treinamento

In [14]:
# SVM
model_SVM.fit(x_train, y_train)

# Random Forest
model_RF.fit(x_train, y_train)

# Logistic Regression
model_LR.fit(x_train, y_train)

LogisticRegression()

### Mapeando pesos iniciais

In [15]:
# SVM
SVM_weights = model_SVM.coef_

# RandomForest
RF_weights = model_RF.feature_importances_

# Logistic Regression
LR_weights = model_LR.coef_

Maiores e menores valores

In [16]:
# SVM
SVM_max = SVM_weights.max()
SVM_min = SVM_weights.min()

# RandomForest
RF_max = RF_weights.max()
RF_min = RF_weights.min()

# Logistic Regression
LR_max = LR_weights.max()
LR_min = LR_weights.min()

Função para normalizar os dados entre 0 e 1

In [17]:
# Vetores que irão contem os pesos normalizados das características
SVM_array = []
RF_array = []
LR_array = []

# FUNÇÕES DE NORMALIZAÇÃO DE PESOS
# SVM
for x in SVM_weights:
    if SVM_max != SVM_min:
        result_SVM = (x - SVM_min) / (SVM_max - SVM_min)
        SVM_array.append(result_SVM)
    else:
        result_SVM = 0.5
        SVM_array.append(result_SVM)

# RandomForest
for x in RF_weights:
    if RF_max != RF_min:
        result_RF = (x - RF_min) / (RF_max - RF_min)
        RF_array.append(result_RF)
    else:
        result_RF = 0.5
        RF_array.append(result_RF)

# LR
for x in LR_weights:
    if LR_max != LR_min:
        result_LR = (x - LR_min) / (LR_max - LR_min)
        LR_array.append(result_LR)
    else:
        result_LR = 0.5
        LR_array.append(result_LR)

Média dos pesos dos três modelos

In [18]:
init_weights = (SVM_array[0] + RF_array[0] + LR_array[0]) / 3

In [19]:
init_weights

array([0.33714209, 0.44890448, 0.34041645, 0.64044489, 0.48626084,
       0.46696487, 0.83028964, 0.33525   , 0.49962579, 0.36035641,
       0.41171017, 0.4958184 , 0.69102066, 0.85374403, 0.47904646,
       0.2877165 , 0.47547093, 0.62379097, 0.42405069, 0.39422559,
       0.43765691, 0.32490819, 0.60079994, 0.19892221, 0.55918927,
       0.56725453, 0.5144677 , 0.63509361])

## Funções de Mapeamento de Pesos

Utilizamos 5 funções de mapeamento de pesos: Potência, Exponencial, Logarítmica, Hiperbólica e Curva em S.

In [20]:
init_weights.max()

0.8537440342494543

In [21]:
def power_function(v):
    a, y, x = v
    result = y * pow(x, exp = a)
    return result

def exponential_function(v):
    a, b, y, x = v
    result = y * (pow(a, exp=(b*x)) - 1) / (pow(a, exp=b) - 1)
    return result

def logarithmic_function(v):
    a, y, x = v
    result = y * ((np.log(1 + a*x)) / (np.log(1 + a)))
    return result

def hyperbolic_function(v):
    a, b, y, x = v
    result = (y * ((a*x) / (1 + (b*x)))) / (a / (1 + b))
    return result

def S_curve_function(v):
    a, y, x = v
    result = (y * ((1 / (1 + (a * np.exp(-x)))) - (1 / (1 + a)))) / ((1 / (1 + (a*np.exp(-1)))) - (1 / (1+a)))
    return result

### SVM Parametrizado

#### Testando o modelo SVM

In [22]:
power_weights_SVM = []
exponential_weights_SVM = []
logarithmic_weights_SVM = []
hyperbolic_weights_SVM = []
S_curve_weights_SVM = []

for w in init_weights:
    power_w = power_function([0.161, 0.665, w])
    power_weights_SVM.append(power_w)

    exponential_w = exponential_function([2.792, 6.813, 9.366, w])
    exponential_weights_SVM.append(exponential_w)

    logarithmic_w = logarithmic_function([7.870, 1.423, w])
    logarithmic_weights_SVM.append(logarithmic_w)

    hyperbolic_w = hyperbolic_function([6.532, 9.035, 0.892, w])
    hyperbolic_weights_SVM.append(hyperbolic_w)

    S_curve_w = S_curve_function([3.929, 1.841, w])
    S_curve_weights_SVM.append(S_curve_w)

#### Função para podar features que tenham peso maior que o peso máximo do init_weights

In [23]:
drebin_power = pd.DataFrame()
drebin_exponential = pd.DataFrame()
drebin_logarithmic = pd.DataFrame()
drebin_hyperbolic = pd.DataFrame()
drebin_S_curve = pd.DataFrame()

In [24]:
max = init_weights.max()
max

0.8537440342494543

#### Função de Potência

In [25]:
count = 0
count_aux_power = []
for weight in power_weights_SVM:
    if weight >= 0 and weight <= max:
        count_aux_power.append(count)
    else:
        print('')
    count += 1

In [26]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_power:
        if count == x:
            drebin_power[col] = drebin[col]
            drebin_power.replace(1, power_weights_SVM[x], inplace=True)
    count+=1

#### Função Exponencial

In [27]:
count = 0
count_aux_exponential = []
for weight in exponential_weights_SVM:
    if weight >= 0 and weight <= max:
        count_aux_exponential.append(count)
    else:
        ''
    count += 1

In [28]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_exponential:
        if count == x:
            drebin_exponential[col] = drebin[col]
            drebin_exponential.replace(1, exponential_weights_SVM[x], inplace=True)
    count+=1

#### Função Logarítmica

In [29]:
count = 0
count_aux_logarithmic = []
for weight in logarithmic_weights_SVM:
    if weight >= 0 and weight <= max:
        count_aux_logarithmic.append(count)
    else:
        ''
    count += 1

In [30]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_logarithmic:
        if count == x:
            drebin_logarithmic[col] = drebin[col]
            drebin_logarithmic.replace(1, logarithmic_weights_SVM[x], inplace=True)
    count+=1

#### Função Hiperbólica

In [31]:
count = 0
count_aux_hyperbolic = []
for weight in hyperbolic_weights_SVM:
    if weight >= 0 and weight <= max:
        count_aux_hyperbolic.append(count)
    else:
        ''
    count += 1

In [32]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_hyperbolic:
        if count == x:
            drebin_hyperbolic[col] = drebin[col]
            drebin_hyperbolic.replace(1, hyperbolic_weights_SVM[x], inplace=True)
    count+=1

#### Função Curva_S

In [33]:
count = 0
count_aux_S_curve = []
for weight in S_curve_weights_SVM:
    if weight >= 0 and weight <= max:
        count_aux_S_curve.append(count)
    else:
        ''
    count += 1

In [34]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_S_curve:
        if count == x:
            drebin_S_curve[col] = drebin[col]
            drebin_S_curve.replace(1, S_curve_weights_SVM[x], inplace=True)
    count+=1

#### Shapes

In [35]:
print("Drebin >>", drebin.shape)
print("\nDrebin_Power >>", drebin_power.shape)
print("Drebin_Exponential >>", drebin_exponential.shape)
print("Drebin_Logarithmic >>", drebin_logarithmic.shape)
print("Drebin_Hyperbolic >>", drebin_hyperbolic.shape)
print("Drebin_S_Curve >>", drebin_S_curve.shape)

Drebin >> (15036, 215)

Drebin_Power >> (15036, 28)
Drebin_Exponential >> (15036, 25)
Drebin_Logarithmic >> (15036, 6)
Drebin_Hyperbolic >> (15036, 25)
Drebin_S_Curve >> (15036, 18)


In [36]:
drebin_power['class'] = drebin['class']
drebin_exponential['class'] = drebin['class']
drebin_logarithmic['class'] = drebin['class']
drebin_hyperbolic['class'] = drebin['class']
drebin_S_curve['class'] = drebin['class']

#### Treinando modelos com features selecionadas

Variáveis preditoras e alvo

In [37]:
# Power
x_power = drebin_power.drop(columns=['class'])
y_power = drebin_power['class']

# Exponential
x_exponential = drebin_exponential.drop(columns=['class'])
y_exponential = drebin_exponential['class']

# Logarithmic
x_logarithmic = drebin_logarithmic.drop(columns=['class'])
y_logarithmic = drebin_logarithmic['class']

# Hyperbolic
x_hyperbolic = drebin_hyperbolic.drop(columns=['class'])
y_hyperbolic = drebin_hyperbolic['class']

# S_Curve
x_S_curve = drebin_S_curve.drop(columns=['class'])
y_S_curve = drebin_S_curve['class']

Variáveis de treino e teste

In [38]:
x_train_power, x_test_power, y_train_power, y_test_power = train_test_split(x_power, y_power, train_size=0.8)
x_train_exponential, x_test_exponential, y_train_exponential, y_test_exponential = train_test_split(x_exponential, y_exponential, train_size=0.8)
x_train_logarithmic, x_test_logarithmic, y_train_logarithmic, y_test_logarithmic = train_test_split(x_logarithmic, y_logarithmic, train_size=0.8)
x_train_hyperbolic, x_test_hyperbolic, y_train_hyperbolic, y_test_hyperbolic = train_test_split(x_hyperbolic, y_hyperbolic, train_size=0.8)
x_train_S_curve, x_test_S_curve, y_train_S_curve, y_test_S_curve = train_test_split(x_S_curve, y_S_curve, train_size=0.8)

#### Modelos SVM

In [39]:
model_SVM_power = SVC()
model_SVM_exponential = SVC()
model_SVM_logarithmic = SVC()
model_SVM_hyperbolic = SVC()
model_SVM_S_curve = SVC()

Treinamento dos modelos

In [40]:
model_SVM_power.fit(x_train_power, y_train_power)
model_SVM_exponential.fit(x_train_exponential, y_train_exponential)
model_SVM_logarithmic.fit(x_train_logarithmic, y_train_logarithmic)
model_SVM_hyperbolic.fit(x_train_hyperbolic, y_train_hyperbolic)
model_SVM_S_curve.fit(x_train_S_curve, y_train_S_curve)

SVC()

#### Métricas

In [41]:
# Power
print("Acurácia >> ", (cross_val_score(model_SVM_power, x_test_power, y_test_power, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_power, x_test_power, y_test_power, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_power, x_test_power, y_test_power, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_power, x_test_power, y_test_power, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  92.81933212088379 %
Precisão >>  90.71835419506907 %
Recall >>  89.52843503528435 %
F1-Score >>  90.07678650027691 %


In [42]:
# Exponential
print("Acurácia >> ", (cross_val_score(model_SVM_exponential, x_test_exponential, y_test_exponential, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_exponential, x_test_exponential, y_test_exponential, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_exponential, x_test_exponential, y_test_exponential, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_exponential, x_test_exponential, y_test_exponential, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  88.03124360838248 %
Precisão >>  85.65305868749513 %
Recall >>  80.42151101340211 %
F1-Score >>  82.94904424379756 %


In [43]:
# Logarithmic
print("Acurácia >> ", (cross_val_score(model_SVM_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  78.12433319882146 %
Precisão >>  64.01271658213955 %
Recall >>  92.76588806000572 %
F1-Score >>  75.74161150712555 %


In [44]:
# Hyperbolic
print("Acurácia >> ", (cross_val_score(model_SVM_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  89.76047672483844 %
Precisão >>  89.26495613873116 %
Recall >>  81.75259443752594 %
F1-Score >>  85.3328018249114 %


In [45]:
# S_curve
print("Acurácia >> ", (cross_val_score(model_SVM_S_curve, x_test_S_curve, y_test_S_curve, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_S_curve, x_test_S_curve, y_test_S_curve, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_S_curve, x_test_S_curve, y_test_S_curve, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_S_curve, x_test_S_curve, y_test_S_curve, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  82.38041801869531 %
Precisão >>  71.48616765119323 %
Recall >>  86.67256273138626 %
F1-Score >>  78.32460321812039 %


In [46]:
drebin_power.head()

,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,android.permission.SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,android.telephony.SmsManager,android.permission.READ_PHONE_STATE,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Ljava.lang.Class.getField,Landroid.content.Context.unregisterReceiver,android.permission.GET_ACCOUNTS,android.permission.RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,android.permission.READ_SMS,getCallingUid,Ljavax.crypto.spec.SecretKeySpec,android.intent.action.BOOT_COMPLETED,android.permission.USE_CREDENTIALS,android.permission.MANAGE_ACCOUNTS,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.589982,0.000000,0.57921,0.0,0.000000,0.0000,0.000000,0.0,0.00000,0.606988,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.589982,0.000000,0.57921,0.0,0.582162,0.0000,0.612628,0.0,0.00000,0.606988,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.000000,0.616344,0.00000,0.0,0.000000,0.0000,0.000000,0.0,0.00000,0.000000,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.564229,0.576462,0.0,0.000000,0.648284,0.0,0.000000,0.589982,0.000000,0.00000,0.0,0.000000,0.5549,0.612628,0.0,0.60559,0.606988,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.576462,0.0,0.000000,0.648284,0.0,0.544145,0.000000,0.000000,0.00000,0.0,0.000000,0.5549,0.000000,0.0,0.00000,0.606988,0.0,0.0,1


In [47]:
drebin_S_curve.head()

,transact,onServiceConnected,bindService,ServiceConnection,android.os.Binder,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Landroid.content.Context.unregisterReceiver,android.permission.GET_ACCOUNTS,android.permission.RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,getCallingUid,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.78293,0.689266,0.0,0.000000,0.000000,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.78293,0.689266,0.0,0.713848,0.000000,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.000000,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.576154,0.667097,0.0,0.0,0.00000,0.78293,0.000000,0.0,0.000000,0.514628,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.667097,0.0,0.0,0.45119,0.00000,0.000000,0.0,0.000000,0.514628,0.0,1


### SVM Original

In [48]:
# criando modelo
model_SVM_original = SVC()

# trainando modelo
model_SVM_original.fit(x_train, y_train)

# métricas modelo
print("Acurácia >> ", (cross_val_score(model_SVM_original, x_test, y_test, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_original, x_test, y_test, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_original, x_test, y_test, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_original, x_test, y_test, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.11872239512219 %
Precisão >>  91.41113411183304 %
Recall >>  89.4495412844037 %
F1-Score >>  90.4007217073462 %


In [49]:
drebin_power.head()

,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,android.permission.SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,android.telephony.SmsManager,android.permission.READ_PHONE_STATE,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Ljava.lang.Class.getField,Landroid.content.Context.unregisterReceiver,android.permission.GET_ACCOUNTS,android.permission.RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,android.permission.READ_SMS,getCallingUid,Ljavax.crypto.spec.SecretKeySpec,android.intent.action.BOOT_COMPLETED,android.permission.USE_CREDENTIALS,android.permission.MANAGE_ACCOUNTS,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.589982,0.000000,0.57921,0.0,0.000000,0.0000,0.000000,0.0,0.00000,0.606988,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.589982,0.000000,0.57921,0.0,0.582162,0.0000,0.612628,0.0,0.00000,0.606988,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.645383,0.0,0.0,0.000000,0.000000,0.0,0.626585,0.648284,0.0,0.000000,0.000000,0.616344,0.00000,0.0,0.000000,0.0000,0.000000,0.0,0.00000,0.000000,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.564229,0.576462,0.0,0.000000,0.648284,0.0,0.000000,0.589982,0.000000,0.00000,0.0,0.000000,0.5549,0.612628,0.0,0.60559,0.606988,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.576462,0.0,0.000000,0.648284,0.0,0.544145,0.000000,0.000000,0.00000,0.0,0.000000,0.5549,0.000000,0.0,0.00000,0.606988,0.0,0.0,1


### Logistic Regression Original

In [50]:
# criando modelo
model_LR_original = LogisticRegression()

# treinando modelo
model_LR_original.fit(x_train, y_train)

# métricas modelo
print("Acurácia >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  91.45659780764063 %
Precisão >>  87.9998511035386 %
Recall >>  88.53211009174314 %
F1-Score >>  88.24904501628221 %


## Algoritmo de Evolução Diferencial (ED)

Utilizamos o algoritmo ED para otimizar em conjunto os parâmetros da função de mapeamento de peso e do classificador final.

In [51]:
from scipy.optimize import differential_evolution

### Função de Potência

In [52]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(power_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_power = result['x']
evaluation = power_function(solution_power)
print('Solution_power: f(%s) = %.5f' % (solution_power, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 5.24883e-22
differential_evolution step 2: f(x)= 5.24883e-22
differential_evolution step 3: f(x)= 3.63957e-26
differential_evolution step 4: f(x)= 1.19724e-28
differential_evolution step 5: f(x)= 1.04443e-29
differential_evolution step 6: f(x)= 1.04443e-29
differential_evolution step 7: f(x)= 1.04443e-29
differential_evolution step 8: f(x)= 1.04443e-29
differential_evolution step 9: f(x)= 6.18501e-30
differential_evolution step 10: f(x)= 3.22298e-30
differential_evolution step 11: f(x)= 1.17696e-38
differential_evolution step 12: f(x)= 2.07954e-39
differential_evolution step 13: f(x)= 1.15057e-39
differential_evolution step 14: f(x)= 1.08397e-39
differential_evolution step 15: f(x)= 1.08397e-39
differential_evolution step 16: f(x)= 3.28457e-55
differential_evolution step 17: f(x)= 3.28457e-55
differential_evolution step 18: f(x)= 2.04655e-56
differential_evolution step 19: f(x)= 1.15437e-56
differential_evolution step 20: f(x)= 3.39189e-58
different

### Função Exponencial

In [53]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(exponential_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_exponential = result['x']
evaluation = exponential_function(solution_exponential)
print('Solution_exponential: f(%s) = %.5f' % (solution_exponential, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 5.15961e-12
differential_evolution step 2: f(x)= 5.15961e-12
differential_evolution step 3: f(x)= 2.87923e-12
differential_evolution step 4: f(x)= 2.87923e-12
differential_evolution step 5: f(x)= 2.87923e-12
differential_evolution step 6: f(x)= 1.84756e-12
differential_evolution step 7: f(x)= 1.37163e-12
differential_evolution step 8: f(x)= 1.0752e-12
differential_evolution step 9: f(x)= 1.0752e-12
differential_evolution step 10: f(x)= 1.0752e-12
differential_evolution step 11: f(x)= 1.0752e-12
differential_evolution step 12: f(x)= 1.0752e-12
differential_evolution step 13: f(x)= 9.39575e-13
differential_evolution step 14: f(x)= 9.39575e-13
differential_evolution step 15: f(x)= 9.39575e-13
differential_evolution step 16: f(x)= 9.39575e-13
differential_evolution step 17: f(x)= 4.39207e-14
differential_evolution step 18: f(x)= 4.39207e-14
differential_evolution step 19: f(x)= 4.08319e-14
differential_evolution step 20: f(x)= 4.08319e-14
differential_e

### Função Logarítmica

In [54]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(logarithmic_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_logarithmic = result['x']
evaluation = logarithmic_function(solution_logarithmic)
print('Solution_logarithmic: f(%s) = %.5f' % (solution_logarithmic, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 2.15944e-06
differential_evolution step 2: f(x)= 2.15944e-06
differential_evolution step 3: f(x)= 2.15944e-06
differential_evolution step 4: f(x)= 2.15944e-06
differential_evolution step 5: f(x)= 2.09685e-06
differential_evolution step 6: f(x)= 2.09685e-06
differential_evolution step 7: f(x)= 2.09685e-06
differential_evolution step 8: f(x)= 2.09685e-06
differential_evolution step 9: f(x)= 2.09685e-06
differential_evolution step 10: f(x)= 2.09685e-06
differential_evolution step 11: f(x)= 2.09685e-06
differential_evolution step 12: f(x)= 2.09685e-06
differential_evolution step 13: f(x)= 2.09685e-06
differential_evolution step 14: f(x)= 1.81066e-06
differential_evolution step 15: f(x)= 1.81066e-06
differential_evolution step 16: f(x)= 1.81066e-06
differential_evolution step 17: f(x)= 1.81066e-06
differential_evolution step 18: f(x)= 1.81066e-06
differential_evolution step 19: f(x)= 1.81066e-06
differential_evolution step 20: f(x)= 1.81066e-06
different

### Função Hiperbólica

In [55]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(hyperbolic_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_hyperbolic = result['x']
evaluation = hyperbolic_function(solution_hyperbolic)
print('Solution_hyperbolic: f(%s) = %.5f' % (solution_hyperbolic, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 6.30882e-06
differential_evolution step 2: f(x)= 6.30882e-06
differential_evolution step 3: f(x)= 6.30882e-06
differential_evolution step 4: f(x)= 6.30882e-06
differential_evolution step 5: f(x)= 5.84139e-06
differential_evolution step 6: f(x)= 5.84139e-06
differential_evolution step 7: f(x)= 5.84139e-06
differential_evolution step 8: f(x)= 5.84139e-06
differential_evolution step 9: f(x)= 5.84139e-06
differential_evolution step 10: f(x)= 5.84139e-06
differential_evolution step 11: f(x)= 5.47638e-06
differential_evolution step 12: f(x)= 5.47638e-06
differential_evolution step 13: f(x)= 5.47638e-06
differential_evolution step 14: f(x)= 5.47638e-06
differential_evolution step 15: f(x)= 4.40167e-06
differential_evolution step 16: f(x)= 4.40167e-06
differential_evolution step 17: f(x)= 4.40167e-06
differential_evolution step 18: f(x)= 4.40167e-06
differential_evolution step 19: f(x)= 4.40167e-06
differential_evolution step 20: f(x)= 4.40167e-06
different

### Função de Curva em S

In [56]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(S_curve_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_S_curve = result['x']
evaluation = S_curve_function(solution_S_curve)
print('Solution_S_curve: f(%s) = %.5f' % (solution_S_curve, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 3.28687e-06
differential_evolution step 2: f(x)= 3.28687e-06
differential_evolution step 3: f(x)= 3.28687e-06
differential_evolution step 4: f(x)= 3.28687e-06
differential_evolution step 5: f(x)= 3.28687e-06
differential_evolution step 6: f(x)= 1.68133e-06
differential_evolution step 7: f(x)= 1.68133e-06
differential_evolution step 8: f(x)= 1.68133e-06
differential_evolution step 9: f(x)= 1.68133e-06
differential_evolution step 10: f(x)= 8.93472e-07
differential_evolution step 11: f(x)= 8.93472e-07
differential_evolution step 12: f(x)= 8.93472e-07
differential_evolution step 13: f(x)= 8.93472e-07
differential_evolution step 14: f(x)= 8.93472e-07
differential_evolution step 15: f(x)= 8.93472e-07
differential_evolution step 16: f(x)= 8.93472e-07
differential_evolution step 17: f(x)= 6.96932e-07
differential_evolution step 18: f(x)= 6.96932e-07
differential_evolution step 19: f(x)= 6.96932e-07
differential_evolution step 20: f(x)= 3.03774e-08
different

## Testando Otimização de parâmetros

In [57]:
power_weights_SVM_optimized = []
exponential_weights_SVM_optimized = []
logarithmic_weights_SVM_optimized = []
hyperbolic_weights_SVM_optimized = []
S_curve_weights_SVM_optimized = []

for w in init_weights:
    power_w = power_function(solution_power)
    power_weights_SVM_optimized.append(power_w)

    exponential_w = exponential_function(solution_exponential)
    exponential_weights_SVM_optimized.append(exponential_w)

    logarithmic_w = logarithmic_function(solution_logarithmic)
    logarithmic_weights_SVM_optimized.append(logarithmic_w)

    hyperbolic_w = hyperbolic_function(solution_hyperbolic)
    hyperbolic_weights_SVM_optimized.append(hyperbolic_w)

    S_curve_w = S_curve_function(solution_S_curve)
    S_curve_weights_SVM_optimized.append(S_curve_w)

In [58]:
drebin_power_optimized = pd.DataFrame()
drebin_exponential_optimized = pd.DataFrame()
drebin_logarithmic_optimized = pd.DataFrame()
drebin_hyperbolic_optimized = pd.DataFrame()
drebin_S_curve_optimized = pd.DataFrame()

### Função de Potência

In [59]:
count = 0
count_aux_power_optimized = []
for weight in power_weights_SVM_optimized:
    if weight >= 0 and weight <= max:
        count_aux_power_optimized.append(count)
    else:
        print('')
    count += 1

In [60]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_power_optimized:
        if count == x:
            drebin_power_optimized[col] = drebin[col]
            drebin_power_optimized.replace(1, power_weights_SVM_optimized[x], inplace=True)
    count+=1

### Função Exponencial

In [61]:
count = 0
count_aux_exponential_optimized = []
for weight in exponential_weights_SVM_optimized:
    if weight >= 0 and weight <= max:
        count_aux_exponential_optimized.append(count)
    else:
        ''
    count += 1

In [62]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_exponential_optimized:
        if count == x:
            drebin_exponential_optimized[col] = drebin[col]
            drebin_exponential_optimized.replace(1, exponential_weights_SVM_optimized[x], inplace=True)
    count+=1

### Função Logarítmica

In [63]:
count = 0
count_aux_logarithmic_optimized = []
for weight in logarithmic_weights_SVM_optimized:
    if weight >= 0 and weight <= max:
        count_aux_logarithmic_optimized.append(count)
    else:
        ''
    count += 1

In [64]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_logarithmic_optimized:
        if count == x:
            drebin_logarithmic_optimized[col] = drebin[col]
            drebin_logarithmic_optimized.replace(1, logarithmic_weights_SVM_optimized[x], inplace=True)
    count+=1

### Função Hiperbólica

In [65]:
count = 0
count_aux_hyperbolic_optimized = []
for weight in hyperbolic_weights_SVM_optimized:
    if weight >= 0 and weight <= max:
        count_aux_hyperbolic_optimized.append(count)
    else:
        ''
    count += 1

In [66]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_hyperbolic_optimized:
        if count == x:
            drebin_hyperbolic_optimized[col] = drebin[col]
            drebin_hyperbolic_optimized.replace(1, hyperbolic_weights_SVM_optimized[x], inplace=True)
    count+=1

### Função de Forma de S em Curva

In [67]:
count = 0
count_aux_S_curve_optimized = []
for weight in S_curve_weights_SVM_optimized:
    if weight >= 0 and weight <= max:
        count_aux_S_curve_optimized.append(count)
    else:
        ''
    count += 1

In [68]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_S_curve_optimized:
        if count == x:
            drebin_S_curve_optimized[col] = drebin[col]
            drebin_S_curve_optimized.replace(1, S_curve_weights_SVM_optimized[x], inplace=True)
    count+=1

### Shapes

In [69]:
print("Drebin >>", drebin.shape)
print("\nDrebin_Power >>", drebin_power_optimized.shape)
print("Drebin_Exponential >>", drebin_exponential_optimized.shape)
print("Drebin_Logarithmic >>", drebin_logarithmic_optimized.shape)
print("Drebin_Hyperbolic >>", drebin_hyperbolic_optimized.shape)
print("Drebin_S_Curve >>", drebin_S_curve_optimized.shape)

Drebin >> (15036, 215)

Drebin_Power >> (15036, 28)
Drebin_Exponential >> (15036, 28)
Drebin_Logarithmic >> (15036, 28)
Drebin_Hyperbolic >> (15036, 28)
Drebin_S_Curve >> (15036, 28)


In [70]:
drebin_power_optimized['class'] = drebin['class']
drebin_exponential_optimized['class'] = drebin['class']
drebin_logarithmic_optimized['class'] = drebin['class']
drebin_hyperbolic_optimized['class'] = drebin['class']
drebin_S_curve_optimized['class'] = drebin['class']

#### Treinando modelos com features selecionadas

Variáveis preditoras e alvo

In [71]:
# Power
x_power_optimized = drebin_power_optimized.drop(columns=['class'])
y_power_optimized = drebin_power_optimized['class']

# Exponential
x_exponential_optimized = drebin_exponential_optimized.drop(columns=['class'])
y_exponential_optimized = drebin_exponential_optimized['class']

# Logarithmic
x_logarithmic_optimized = drebin_logarithmic_optimized.drop(columns=['class'])
y_logarithmic_optimized = drebin_logarithmic_optimized['class']

# Hyperbolic
x_hyperbolic_optimized = drebin_hyperbolic_optimized.drop(columns=['class'])
y_hyperbolic_optimized = drebin_hyperbolic_optimized['class']

# S_Curve
x_S_curve_optimized = drebin_S_curve_optimized.drop(columns=['class'])
y_S_curve_optimized = drebin_S_curve_optimized['class']

Variáveis de treino e teste

In [72]:
x_train_power_optimized, x_test_power_optimized, y_train_power_optimized, y_test_power_optimized = train_test_split(x_power_optimized, y_power_optimized, train_size=0.8)
x_train_exponential_optimized, x_test_exponential_optimized, y_train_exponential_optimized, y_test_exponential_optimized = train_test_split(x_exponential_optimized, y_exponential_optimized, train_size=0.8)
x_train_logarithmic_optimized, x_test_logarithmic_optimized, y_train_logarithmic_optimized, y_test_logarithmic_optimized = train_test_split(x_logarithmic_optimized, y_logarithmic_optimized, train_size=0.8)
x_train_hyperbolic_optimized, x_test_hyperbolic_optimized, y_train_hyperbolic_optimized, y_test_hyperbolic_optimized = train_test_split(x_hyperbolic_optimized, y_hyperbolic_optimized, train_size=0.8)
x_train_S_curve_optimized, x_test_S_curve_optimized, y_train_S_curve_optimized, y_test_S_curve_optimized = train_test_split(x_S_curve_optimized, y_S_curve_optimized, train_size=0.8)

#### Modelos SVM

In [73]:
model_SVM_power_optimized = SVC()
model_SVM_exponential_optimized = SVC()
model_SVM_logarithmic_optimized = SVC()
model_SVM_hyperbolic_optimized = SVC()
model_SVM_S_curve_optimized = SVC()

Treinamento dos modelos

In [74]:
model_SVM_power_optimized.fit(x_train_power_optimized, y_train_power_optimized)
model_SVM_exponential_optimized.fit(x_train_exponential_optimized, y_train_exponential_optimized)
model_SVM_logarithmic_optimized.fit(x_train_logarithmic_optimized, y_train_logarithmic_optimized)
model_SVM_hyperbolic_optimized.fit(x_train_hyperbolic_optimized, y_train_hyperbolic_optimized)
model_SVM_S_curve_optimized.fit(x_train_S_curve_optimized, y_train_S_curve_optimized)

SVC()

#### Métricas

In [75]:
# Power
print("Acurácia >> ", (cross_val_score(model_SVM_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.51778044344698 %
Precisão >>  90.97249309410843 %
Recall >>  90.69625298963594 %
F1-Score >>  90.82714300096916 %


In [76]:
# Exponential
print("Acurácia >> ", (cross_val_score(model_SVM_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.35045135184437 %
Precisão >>  91.85904326281685 %
Recall >>  90.1952380952381 %
F1-Score >>  91.00855995145724 %


In [77]:
# Logarithmic
print("Acurácia >> ", (cross_val_score(model_SVM_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.64962051066605 %
Precisão >>  91.42003033267328 %
Recall >>  91.35135135135135 %
F1-Score >>  91.36931151636225 %


In [78]:
# Hyperbolic
print("Acurácia >> ", (cross_val_score(model_SVM_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.15222138075524 %
Precisão >>  91.58003603770393 %
Recall >>  89.8818866111467 %
F1-Score >>  90.69434386601692 %


In [79]:
# S_curve
print("Acurácia >> ", (cross_val_score(model_SVM_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_SVM_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_SVM_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_SVM_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  93.45105886645182 %
Precisão >>  91.62368476215066 %
Recall >>  91.51021605957229 %
F1-Score >>  91.55397206368987 %
